In [1]:
import tensorflow as tf

In [2]:
def testGPU():
    with tf.device('/gpu:0'):
        x = tf.constant(10)
        y = tf.constant(20)
        mul = tf.multiply(x, y)
        
        with tf.Session() as sess:
            res = sess.run(mul)
            print(res)
        
testGPU()

200


# Read TFRecords

In [14]:
def get_tfrecords_features():
    return {
        'image': tf.FixedLenFeature([1], tf.string),
        'label': tf.FixedLenFeature([1], tf.int64)
        }

def feature_retrieval(serialized_example, IMAGE_HEIGHT=3000, IMAGE_WIDTH=3000):
    
    features = tf.parse_single_example(
                  serialized_example,
                  features=get_tfrecords_features()
               )
    # Decoding ...
#     _height = tf.cast(features['height'], tf.int64)[0]
#     _width = tf.cast(features['width'], tf.int64)[0]
    _image_raw = tf.cast(features['image'], tf.string)[0]
    
    # Image processing
    image_shape = tf.stack([IMAGE_HEIGHT, IMAGE_WIDTH, 3])
    # has to be uint8 type
    image_raw = tf.image.decode_jpeg(_image_raw)
    image = tf.reshape(image_raw, image_shape)
    # Important, since we do not know the image shape information, it is all encoded in Tensorflow.
    # Hence, it can hardly pass the shape check in your Tensorflow Neural Network.
    resized_image = tf.image.resize_image_with_crop_or_pad(
        image=image,
        target_height=IMAGE_HEIGHT,
        target_width=IMAGE_WIDTH
    )
    resized_image = tf.cast(resized_image, tf.float32)
    # Label processing
    label = tf.cast(features['label'][0], tf.int64)
    
    return resized_image, label

In [15]:
def load_tfrecords(tfrecords_filepath):
    items = []
    labels = []
    print("Loading %s" % tfrecords_filepath)
    with tf.Session() as sess:
        for serialized_example in tf.python_io.tf_record_iterator(tfrecords_filepath):
            data, label = feature_retrieval(serialized_example)
            items.append(data)
            labels.append(label)
    print("Finished Loading %s" % tfrecords_filepath)
    return (items, labels)

images, labels = load_tfrecords('../data.tfrecords')
with tf.Session() as sess:
    import numpy as np
    print(np.array(images[0].eval()).shape)
    print(labels[0].eval())

Loading ../data.tfrecords
Finished Loading ../data.tfrecords
(3000, 3000, 3)
0


# Feed in

In [5]:
def prepare_data(filename_queue):
    
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    
    image, label = feature_retrieval(serialized_example)
    
    images, labels = tf.train.shuffle_batch(
        [image, label],
        batch_size=2,
        capacity=50,
        num_threads=1,
        min_after_dequeue=10)
    return images, labels

In [6]:
def _get_model(image, output_units):
    # 3000 x 3000 x 3
    conv1 = tf.layers.conv2d(image, 32, 17, (10, 10), activation=tf.nn.relu)
    # 300 x 300 x 32
    pool1 = tf.layers.max_pooling2d(conv1, (5, 5), (5, 5))
    # 60 x 60 x 32
    conv2 = tf.layers.conv2d(pool1, 64, (5, 5), (1, 1), activation=tf.nn.relu)
    # 60 x 60 x 64
    pool2 = tf.layers.max_pooling2d(conv2, (5, 5), (3, 3))
    # 20 x 20 x 64
    conv3 = tf.layers.conv2d(pool2, 128, (5, 5), (1, 1), activation=tf.nn.relu)
    # 20 x 20 x 128
    pool3 = tf.layers.max_pooling2d(conv3, (5, 5), (2, 2))
    # 10 x 10 x 128
    dropout = tf.layers.dropout(pool3)
    flattern = tf.layers.flatten(dropout)
    # 10 x 10 x 128
    dense1 = tf.layers.dense(flattern, output_units, activation=tf.nn.softmax)
    
    return dense1

def get_trainer(images_batch, labels_batch, output_units, learning_rate=0.1):
    # Get model and loss 
    pred = _get_model(images_batch, output_units=output_units)
    print(pred, labels_batch)
    _loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=labels_batch)
    loss = tf.reduce_mean(_loss)

    correct = tf.equal(tf.argmax(pred, 1), tf.to_int64(labels_batch))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(loss)
    
    ops = {
        'pred': pred,
        'loss': loss,
        'accuracy': accuracy,
        'train_op': train_op
    }
    return ops

def get_sess():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    config.log_device_placement = True
    sess = tf.Session(config=config)
    return sess

In [7]:
def train(num_epochs):
    
    tf.reset_default_graph()
    
#     filename_queue = tf.FIFOQueue(num_epochs, tf.string)
#     enqueue_placeholder = tf.placeholder(dtype=tf.string)
#     enqueue_op = filename_queue.enqueue(enqueue_placeholder)
    
#     images_batch, labels_batch = prepare_data(filename_queue)
    
    filename_queue = tf.train.string_input_producer(
            ['../data.tfrecords'], 
            num_epochs=num_epochs
        )
    images_batch, labels_batch = prepare_data(filename_queue)
    ops = get_trainer(images_batch, labels_batch, 5)
    
    init_op = tf.group(
        tf.global_variables_initializer(),
        # if we have 'num_epochs' in filename_queue
        tf.local_variables_initializer()
    )
    
    with get_sess() as sess:
        sess.run(init_op)
#         for i in range(num_epochs):
# #             sess.run([enqueue_op], feed_dict={enqueue_placeholder:"../data.tfrecords"})
        train_one_epoch(sess, ops)
    
def train_one_epoch(sess, ops):
    # Start training
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    count = 0
    while not coord.should_stop():
        try:
            _, _loss, _acc = sess.run([ops['train_op'], ops['loss'], ops['accuracy']])
            count += 1
            print(_acc, _loss)
            if count == 5:
                # For test set
                pass
        except tf.errors.OutOfRangeError:
            print('Finished one epoch')
            break
            
    coord.request_stop()
    coord.join(threads)

In [8]:
# with tf.device('/gpu:0'):
train(120)
tf.reset_default_graph()

Tensor("dense/Softmax:0", shape=(2, 5), dtype=float32) Tensor("shuffle_batch:1", shape=(2,), dtype=int64)
0.0 1.9017876
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.5 1.4048324
0.5 1.4048324
0.5 1.4048324
0.5 1.4048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
0.5 1.4048324
0.5 1.4048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
0.0 1.9048324
1.0 0.90483236
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
1.0 0.90483236
0.0 1.9048324
0.0 1.9048324
0.5 1.4048324
0.5 1.4048324
0.5 1.4048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
0.0 1.9048324
0.5 1.4048

0.5 1.4048324
0.5 1.4048324
0.5 1.4048324
0.0 1.9048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
1.0 0.90483236
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.0 1.9048324
0.5 1.4048324
0.5 1.4048324
0.0 1.9048324
0.5 1.4048324
0.0 1.9048324
Finished one epoch
